In [ ]:
import time
import pandas as pd
from watchdog.observers import Observer
from watchdog.events import PatternMatchingEventHandler

DIRECTORY = './hoge/'
FILE = 'f1.txt'
PATH = DIRECTORY + FILE

len_df = -1

class TextFileEventHandler(PatternMatchingEventHandler):
    def __init__(self, patterns=['*.txt'], ignore_patterns=None, ignore_directories=True, case_sensitive=False):
        super().__init__(patterns, ignore_patterns, ignore_directories, case_sensitive)
   
    # 監視対象のファイルに変更があった時に実行
    def on_modified(self, event):
        # 指定のファイル以外は弾いておく
        if event.src_path[-len(FILE):] != FILE:
            return
        
        bef_len_df = len_df
        df = save_len_df()
        print_df(df=df, start=bef_len_df)

# ファイルの行数を保存
def save_len_df():
    df = pd.read_csv(PATH, header=None)
    len_df = len(df)
    return df

def print_df(df, start):
    print(df[start:].to_string(index=False, header=None))

def main():
    # ファイル内容すべて出力, 行番号取得
    df = save_len_df()
    print(df.to_string(index=False, header=None))

    # 監視
    observer = Observer()
    observer.schedule(TextFileEventHandler(), DIRECTORY, recursive = False)    
    observer.start()

    try:
        while True:
            time.sleep(5)
    except KeyboardInterrupt:
        observer.unschedule_all()
        observer.stop()

main()